目标：
最大熵模型，首先的目的是为了保持最多的不确定性，使最大熵这个等式的值最大，这个时一开始的原理部分，我们如何应用呢，使用的时p(y|x)当给定一个x时求当前属于哪个类别y的概率值最大。首先找到了一些约束条件，然后使用拉格朗日乘法求，最终得到了p(y|x)和w之间的关系，我们的目标进一步发生了转变，什么样的w能好用呢，这时候就可以用多种办法了，比如梯度下降、拟牛顿法和改进的尺度迭代算法。

上面的目标介绍了相关的流程，在程序中有什么用呢，程序只需要两个东西，一个是p(y|x)和w之间的关系，另外一个是优化算法如何更新w的，这样就有了w和p(y|x)之间确定的对应关系，之后再来测试数据就可以直接结算了。

??  感觉实现的有问题，x,y到底是输入的哪一个，是单个还是多个

In [378]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score,confusion_matrix
import pandas as pd
from tqdm import tqdm

In [379]:
def read_file(file_name):
    with open(file_name,'r',encoding='utf-8') as f:
        data = f.readlines()
    label_map = {}
    label_map["ham"] = 0
    label_map["spam"] = 1
    
    
    text = list()
    label = list()
    label_num = list()
    for item in data:
        item = item.strip()
        item = item.split(" ")
        item_len = len(item)
        tmp_list = list()
        if(item[1] in ["ham","spam"]):
            label.append(item[1])
            label_num.append(label_map[item[1]])
            for index in range(2,item_len,2):
                tmp_list.append(item[index])
        text.append(" ".join(tmp_list))
    return text,label,label_num

In [380]:
train_text,train_label,train_label_num = read_file("../data/train")
test_text,test_label,test_label_num = read_file("../data/test")

In [381]:
cv = CountVectorizer(min_df = 512,stop_words='english')
part_fit = cv.fit(train_text) # 以部分句子为参考
train_all_count = cv.transform(train_text) # 对训练集所有邮件统计单词个数
test_all_count = cv.transform(test_text) # 对测试集所有邮件统计单词个数

In [382]:
train_data = train_all_count.toarray() 
test_data = test_all_count.toarray()

In [383]:
train_data_split_num = 9000
train_data = train_data[:train_data_split_num]
train_label_num = train_label_num[:train_data_split_num]

In [384]:
#现在训练数据还有标签就准备好了
#train_data train_label_num test_data test_label_num

In [385]:
#统计特征函数，是一个字典的列表，列表的下标代表了第几个特征，里面是x,y对应的元组
m,n  = train_data.shape
feature_dict = [{} for i in range(n)]
feature_num = 0
p_feature = [train_data[:,index].sum() for index in range(n)]
for feature in range(n):
    for index in range(m):
        tmp_tuple = (train_data[index][feature],train_label_num[index])
        if(tmp_tuple in feature_dict[feature]):
            feature_dict[feature][tmp_tuple]+=1
        else:
            feature_dict[feature][tmp_tuple]=1


In [386]:
p_feature = [item/m for item in p_feature]

In [387]:
feature_num = np.array([len(item) for item in feature_dict]).sum()
w = np.array([0]*feature_num,dtype='float32')

In [388]:
#为了将w和feature对应起来，对每一个feature指定一个id
xy2id = {}
index = 0
for feature in range(n):
    for item in feature_dict[feature]:
        xy2id[(feature,item)] = index
        index+=1

In [389]:
#检验两者的长度
len(xy2id),len(w)

(1293, 1293)

In [390]:
#计算pw(y}x)
#这里的输入X是所有的特征，y是标签
#输出是一个数
def pw_xy(i,x,y):#
    #需要计算分子和分母
    #计算分子
    w_sum = 0
    
    if((x,y) in feature_dict[i]):
        id = xy2id[(i,(x,y))]
        w_sum+=w[id]
    molecule = np.exp(w_sum)
    
    y = 1-y
    w_sum = 0
    if((x,y) in feature_dict[i]):
        id = xy2id[(i,(x,y))]
        w_sum+=w[id]
            
    #返回结果
    return molecule/(molecule+np.exp(w_sum))       

In [391]:
#计算Ep_f
#计算所有的个数
def Ep_f(X,y):
    ep_f = [0]*feature_num
    for i in range(n):
        tmp = (X[i],y)
        if(tmp in feature_dict[i]):
            p_xy = feature_dict[i][tmp]/m
            id = xy2id[(i,tmp)]
            ep_f[id]+=p_xy
    return ep_f

In [392]:
#计算前面所有的期望值
def get_all_p(X,y):
    ep_f = [0]*feature_num
    for i in range(n):
        if (X[i],y) in feature_dict[i]:
            id = xy2id[(i,(X[i],y))]
            if(X[i]==1):
                p_x = p_feature[i]
            else:
                p_x = 1-p_feature[i]
            one_pw_yx = pw_xy(i,X[i],y)
            ep_f[id]+=(p_x*one_pw_yx)
    return ep_f

In [456]:
for iter_num in tqdm(range(10)):
    for index in range(m):
        X = train_data[index]
        y = train_label_num[index]
        w += 0.0001*(np.array(get_all_p(X,y),dtype="float32")-np.array(Ep_f(X,y),dtype="float32"))














  0%|                                                                                           | 0/10 [00:00<?, ?it/s]












 10%|████████▏                                                                         | 1/10 [01:40<15:06, 100.73s/it]












 20%|████████████████▍                                                                 | 2/10 [03:21<13:26, 100.76s/it]












 30%|████████████████████████▌                                                         | 3/10 [05:02<11:46, 100.90s/it]












 40%|████████████████████████████████▊                                                 | 4/10 [06:41<10:02, 100.34s/it]












 50%|█████████████████████████████████████████▌                                         | 5/10 [08:20<08:19, 99.91s/it]












 60%|█████████████████████████████████████████████████▊                                 | 6/10 [09:57<06:36, 99.04s/it]












 70%|██████████████████████████████████████████████████████████

In [501]:
def predict(X):
    zero_num = 0
    one_num = 0
    for index,item in enumerate(X):
        z_n = pw_xy(index,item,0)
        o_n = pw_xy(index,item,1)
#         print(z_n,o_n)
        if(z_n>o_n):
            zero_num+=0.076
        else:
            one_num +=0.924
        
    if(zero_num<one_num):
        return 0
    return 1

In [489]:
w

array([-1.1522831 ,  0.8929158 , -1.6823598 , ...,  4.1836505 ,
       -6.9901195 ,  0.01331823], dtype=float32)

In [502]:
result = list()
for item in tqdm(test_data):
    result.append(predict(item))














  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]












  0%|▍                                                                                | 5/1000 [00:00<00:20, 48.57it/s]












  1%|▊                                                                               | 10/1000 [00:00<00:20, 48.15it/s]












  2%|█▏                                                                              | 15/1000 [00:00<00:20, 47.59it/s]












  2%|█▌                                                                              | 19/1000 [00:00<00:21, 44.88it/s]












  2%|█▉                                                                              | 24/1000 [00:00<00:22, 44.11it/s]












  3%|██▏                                                                             | 28/1000 [00:00<00:25, 37.93it/s]












  3%|██▌                                                       

 24%|██████████████████▉                                                            | 240/1000 [00:05<00:15, 49.19it/s]












 24%|███████████████████▎                                                           | 245/1000 [00:05<00:15, 47.88it/s]












 25%|███████████████████▊                                                           | 250/1000 [00:05<00:15, 47.95it/s]












 26%|████████████████████▏                                                          | 255/1000 [00:05<00:15, 47.72it/s]












 26%|████████████████████▌                                                          | 260/1000 [00:05<00:15, 46.89it/s]












 26%|████████████████████▉                                                          | 265/1000 [00:05<00:15, 46.72it/s]












 27%|█████████████████████▎                                                         | 270/1000 [00:06<00:15, 47.53it/s]












 28%|█████████████████████▋                                                 

 50%|███████████████████████████████████████▎                                       | 498/1000 [00:10<00:14, 35.16it/s]












 50%|███████████████████████████████████████▋                                       | 502/1000 [00:11<00:13, 35.71it/s]












 51%|████████████████████████████████████████                                       | 507/1000 [00:11<00:12, 38.61it/s]












 51%|████████████████████████████████████████▌                                      | 513/1000 [00:11<00:11, 42.33it/s]












 52%|████████████████████████████████████████▉                                      | 518/1000 [00:11<00:10, 44.26it/s]












 52%|█████████████████████████████████████████▎                                     | 523/1000 [00:11<00:10, 45.72it/s]












 53%|█████████████████████████████████████████▊                                     | 529/1000 [00:11<00:09, 48.29it/s]












 53%|██████████████████████████████████████████▏                            

 74%|██████████████████████████████████████████████████████████▏                    | 737/1000 [00:16<00:05, 46.79it/s]












 74%|██████████████████████████████████████████████████████████▌                    | 742/1000 [00:16<00:05, 45.50it/s]












 75%|███████████████████████████████████████████████████████████                    | 747/1000 [00:16<00:05, 45.62it/s]












 75%|███████████████████████████████████████████████████████████▍                   | 753/1000 [00:16<00:05, 47.64it/s]












 76%|███████████████████████████████████████████████████████████▉                   | 759/1000 [00:16<00:04, 48.93it/s]












 76%|████████████████████████████████████████████████████████████▎                  | 764/1000 [00:16<00:04, 48.68it/s]












 77%|████████████████████████████████████████████████████████████▊                  | 769/1000 [00:16<00:04, 47.40it/s]












 77%|█████████████████████████████████████████████████████████████▏         

 98%|█████████████████████████████████████████████████████████████████████████████▎ | 978/1000 [00:21<00:00, 47.21it/s]












 98%|█████████████████████████████████████████████████████████████████████████████▋ | 983/1000 [00:21<00:00, 46.81it/s]












 99%|██████████████████████████████████████████████████████████████████████████████ | 988/1000 [00:21<00:00, 46.53it/s]












 99%|██████████████████████████████████████████████████████████████████████████████▍| 993/1000 [00:21<00:00, 46.60it/s]












100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:21<00:00, 46.27it/s]


In [477]:
# result

In [503]:
accuracy_score(test_label_num,result)

0.914